In [1]:
import torch
from d2l import torch as d2l

**多输入通道**   
卷积核形状如：$c_i\times k_h\times k_w$，通道数为$c_i$，高度为$k_h$，宽度为$h_w$  
输出是一个通道，计算方法是把所有通道相加

In [10]:
def corr2d_multi_in(X,K):
    # 把所有通道的结果加起来
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [11]:
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                  [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],[[1.0,2.0],[3.0,4.0]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

**多输出通道**   
在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。  
直观地说，我们可以将每个通道看作对不同特征的响应。  
而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。  

输入通道数$c_i$，输出通道数为$c_o$，则卷积核形状为：$$c_o\times c_i\times k_h\times k_w$$ 注意$c_o$在前，$c_i$在后。对于每个输出通道，都有一个形状如$c_i\times k_h\times k_w$的卷积核

In [12]:
def corr2d_multi_in_out(X,K):
    # torch.stack()将一系列的张量沿着新的维度进行堆叠起来
    # 注意corr2d_multi_in()的计算中使用了zip()函数，
    # 因为全部的输入通道都与相同的卷积核进行互相关运算来算出单个输出通道
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [13]:
K = torch.stack((K,K+3,K+5),0)
K.shape

torch.Size([3, 2, 2, 2])

In [14]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[116., 156.],
         [236., 276.]],

        [[156., 212.],
         [324., 380.]]])

**1 X 1 卷积层**  
1 X 1 卷积层**失去**了在高度和宽度维度上，识别相邻元素见相互作用的能力  
但是它可以**增加通道数**  
对于通道数为$c_i$的输入，应用$c_o\times c_i$卷积层可以使输出通道数转变为$c_o$

In [15]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]
    # 先将X和K的通道数转换为二维数组
    X = X.reshape((c_i,h*w))
    K = K.reshape((c_o,c_i))
    # 再使用全连接层的矩阵乘法
    Y = torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

In [17]:
X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))

In [18]:
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum())<1e-6